In [7]:
import os
os.chdir("..")
print(os.getcwd())

q:\AD project


## TO-DOS for today
1. Build the vector database
2. Build the basic RAG pipeline
3. HyDE and Modified HyDE integrations

## BUILD VECTOR DATABASE

## GET THE DATA

In [8]:
# BOOK DATA
from src.book_preprocess import get_book_data

book_doc_data = get_book_data(100)

100%|██████████| 10/10 [00:00<00:00, 16.09it/s]


In [9]:
len(book_doc_data)

3040

In [10]:
book_doc_data[0]

{'text': 'Let me begin this preface with a confession of a few of my own biases.\nFirst, I believe that theory, and the models that flow from it, should provide us with the tools to understand, analyze and solve problems.\nThe test of a model or theory then should not be based upon its elegance but upon its usefulness in problem solving.\nSecond, there is little in corporate financial theory, in my view, that is new and revolutionary.\nThe core principles of corporate finance are common sense ones, and have changed little over time.\nThat should not be surprising.\nCorporate finance is only a few decades old and people have been running businesses for thousands of years, and it would be exceedingly presumptuous of us to believe that they were in the dark until corporate finance theorists came along and told them what to do.\nTo be fair, it is true that corporate financial theory has made advances in taking common sense principles and providing them with structure, but these advances ha

In [11]:
import json

with open('artifacts\YouTube_API_Transcripts\chunked_transcripts_mba.json', 'r') as file:
    mba_data = json.load(file)

with open('artifacts\YouTube_API_Transcripts\chunked_transcripts_undergrad.json', 'r') as file:
    undergrad_data = json.load(file)

with open('artifacts\YouTube_API_Transcripts\chunked_misc_transcripts.json', 'r') as file:
    misc_data = json.load(file)

In [12]:
mba_data.keys()

dict_keys(['2S_vt-N0czE', '5T2PmzsxhkM', 'lnPfpvUJEIc', '44Gv-gJxAeo', 'BA3SbC9cFIo', 't3M26iDiEfo', 'iYMsQeChsp4', 'QKQtAGmmP-4', 'eBDAnoxRKIg', 'h8AnJ3deIto', 'H5jLvqp4vas', 's4khp86YR_I', 'vXxN6A88SA4', 'CBM5iXS2pNE', 'flZcxlRtxhg', 'o7HirKysCKQ', 'F3RBKPkDQHY', 'muu0lpWns-0', 'O-sQYbULzwg', 'IFWkYXBzcpw', 'X5uFWJljqYk', 'XNQiZjoZcqY', 'gzJq6bbW-Kk', 'DQDCQxyxJ9U', 'veCrq5J4MyY', '9kid8pL8YDo'])

In [ ]:
from llama_index.schema import Document

all_data_list = []

curr_book = ""
for book_doc in book_doc_data:
    # print(book_doc)
    book = book_doc['book_source']
    if book!= curr_book:
        curr_book = book
        all_data_list.append(
            Document(
                text=book_doc['text'],
                extra_info={
                    'page_num_coordinates':book_doc['page_num_coordinates'],
                    'book_source':book_doc['book_source'],
                }
            )
        )

for json_data in [undergrad_data,mba_data,misc_data]:
    for youtube_id, text_list in json_data.items():
        # print(youtube_id)
        all_data_list.append(
            Document(
                text=text_list[0]['text'],
                extra_info={
                    "youtube_id":youtube_id,
                    "start_timestamp":text_list[0]['start_time'],
                }
            )
        )
        break

In [ ]:
len(all_data_list)

12

In [55]:
all_data_list

[Document(id_='30f3058b-7384-433e-b1b4-25b0f84c6e1e', embedding=None, metadata={'page_num_coordinates': [[{'page_num': 2}, [248.09210205078125, 282.16666666666674], [248.09210205078125, 1008.8333333333334], [1459.0, 1008.8333333333334], [1459.0, 282.16666666666674]]], 'book_source': 'Applied_corporate_finance'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Let me begin this preface with a confession of a few of my own biases.\nFirst, I believe that theory, and the models that flow from it, should provide us with the tools to understand, analyze and solve problems.\nThe test of a model or theory then should not be based upon its elegance but upon its usefulness in problem solving.\nSecond, there is little in corporate financial theory, in my view, that is new and revolutionary.\nThe core principles of corporate finance are common sense ones, and have changed little over time.\nThat should not be surprising.\nCorporate finance is only a few deca

In [57]:
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key = os.environ["OPENAI_API_KEY"]

In [59]:
!pip install -q chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
trulens-eval 0.18.1 requires pydantic<2,>=1.10.7, but you have pydantic 2.6.1 which is incompatible.
trulens-eval 0.18.1 requires typing-extensions==4.5.0, but you have typing-extensions 4.9.0 which is incompatible.
trulens-eval 0.18.1 requires typing-inspect==0.8.0, but you have typing-inspect 0.9.0 which is incompatible.


In [22]:
import chromadb
COLLECTION_NAME = "ad-project"
chroma_client = chromadb.PersistentClient(
    path=COLLECTION_NAME,
)

In [23]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import openai
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

EMBEDDING_MODEL = "text-embedding-3-small"
embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name=EMBEDDING_MODEL)
ad_project_collection = chroma_client.create_collection(name=COLLECTION_NAME,embedding_function=embedding_function)

OPENAI_API_KEY is ready


In [14]:
# !pip install -U llama-index

In [15]:
# !pip install -U langchain

In [13]:
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

all_data_list = []

curr_book = ""
for book_doc in book_doc_data:
    # print(book_doc)
    book = book_doc['book_source']
    if book!= curr_book:
        curr_book = book
        all_data_list.append(
            Document(
                page_content=book_doc['text'],
                metadata={
                    'page_num_coordinates':book_doc['page_num_coordinates'],
                    'book_source':book_doc['book_source'],
                }
            )
        )

for json_data in [undergrad_data,mba_data,misc_data]:
    for youtube_id, text_list in json_data.items():
        # print(youtube_id)
        all_data_list.append(
            Document(
                page_content=text_list[0]['text'],
                metadata={
                    "youtube_id":youtube_id,
                    "start_timestamp":text_list[0]['start_time'],
                }
            )
        )
        break

In [ ]:
# from langchain_community.embeddings import OpenAIEmbeddings

# query = "What are the first steps in Valuation?"
# COLLECTION_NAME = "ad-project"
# embedding_function = OpenAIEmbeddings(model=EMBEDDING_MODEL)
# db2 = Chroma.from_documents(all_data_list, embedding_function, persist_directory=COLLECTION_NAME)
# docs = db2.similarity_search(query)

# # load from disk
# db3 = Chroma(persist_directory=COLLECTION_NAME, embedding_function=embedding_function)
# docs = db3.similarity_search(query)
# print(docs[0].page_content)

ValueError: Expected metadata value to be a str, int, float or bool, got [[{'page_num': 2}, [248.09210205078125, 282.16666666666674], [248.09210205078125, 1008.8333333333334], [1459.0, 1008.8333333333334], [1459.0, 282.16666666666674]]] which is a <class 'list'>

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

In [25]:
!pip3 uninstall llama-index

In [2]:
!pip3 install llama-index --upgrade --no-cache-dir --force-reinstall

     ---------------------------------------- 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
     ---------------------------------------- 83.5/83.5 kB ? eta 0:00:00
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
     ------------------------------------- 247.7/247.7 kB 14.8 MB/s eta 0:00:00
   ---------------------------------------- 15.4/15.4 MB 23.4 MB/s eta 0:00:00
   ---------------------------------------- 2.0/2.0 MB 31.8 MB/s eta 0:00:00
   --------------------------------------- 366.0/366.0 kB 22.2 MB/s eta 0:00:00
   ---------------------------------------- 147.9/147.9 kB ? eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
astroid 2.6.6 requires wrapt<1.13,>=1.11, but you have wrapt 1.16.0 which is incompatible.
botocore 1.24.32 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.
degiro-connector 2.0.21 requires pandas<2.0.0,>=1.1.5, but you have pandas 2.2.0 which is incompatible.
jupyter-server 1.23.6 requires anyio<4,>=3.1.0, but you have anyio 4.3.0 which is incompatible.
langchain 0.0.344 requires anyio<4.0, but you have anyio 4.3.0 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.
openbb

   ---------------------------------------- 170.9/170.9 kB ? eta 0:00:00
   ---------------------------------------- 108.0/108.0 kB ? eta 0:00:00
   ---------------------------------------- 75.9/75.9 kB ? eta 0:00:00
   ---------------------------------------- 77.0/77.0 kB ? eta 0:00:00
   ---------------------------------------- 1.6/1.6 MB 26.0 MB/s eta 0:00:00
   ---------------------------------------- 1.5/1.5 MB 48.4 MB/s eta 0:00:00
   ---------------------------------------- 226.7/226.7 kB ? eta 0:00:00
   ---------------------------------------- 2.6/2.6 MB 23.7 MB/s eta 0:00:00
   ---------------------------------------- 3.4/3.4 MB 27.1 MB/s eta 0:00:00
   ---------------------------------------- 24.5/24.5 MB 18.2 MB/s eta 0:00:00
   --------------------------------------- 284.0/284.0 kB 18.2 MB/s eta 0:00:00
   ---------------------------------------- 152.8/152.8 kB ? eta 0:00:00
   ---------------------------------------- 62.6/62.6 kB ? eta 0:00:00
   -------------------------

In [24]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from IPython.display import Markdown, display
import chromadb

COLLECTION_NAME = "ad-project"
EMBEDDING_MODEL = "text-embedding-3-small"

ad_project_db = chromadb.PersistentClient(path="ad_project_db")
ad_project_chroma_collection = ad_project_db.get_or_create_collection(COLLECTION_NAME)
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)
vector_store = ChromaVectorStore(chroma_collection=ad_project_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    all_data_list, storage_context=storage_context, embed_model=embed_model
)

ImportError: cannot import name 'VectorStoreIndex' from 'llama_index.core' (unknown location)